In [2]:
import numpy as np
import cv2
import os
import keras
from keras.optimizers import Adam
from keras.models import Model
from tensorflow.keras.models import Sequential
from keras import layers
from keras.layers import Dense, Input, BatchNormalization, Activation, Flatten, Dropout, TimeDistributed
from keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, ConvLSTM2D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
def ELA(image_path, quality=90):
    temp_path = "temp.jpg"
    original_image = cv2.imread(image_path)
    cv2.imwrite(temp_path, original_image, [cv2.IMWRITE_JPEG_QUALITY, quality])
    temp_image = cv2.imread(temp_path)
    ELA_image = cv2.absdiff(original_image, temp_image)
    ELA_image = ELA_image.astype(np.uint8)
    os.remove(temp_path)
    return ELA_image

In [4]:
def load(real_dir, fake_dir, img_size = (299, 299)):
    real, fake = [], []
    
    for i in os.listdir(real_dir):
        image_path = os.path.join(real_dir, i)
        image = ELA(image_path)
        image = cv2.resize(image, img_size)
        real.append(image)
        
    for i in os.listdir(fake_dir):
        image_path = os.path.join(fake_dir, i)
        image = ELA(image_path)
        image = cv2.resize(image, img_size)
        fake.append(image)
        
    real_label = [1] * len(real)
    fake_label = [0] * len(fake)
    
    images = np.array(real + fake)
    labels = np.array(real_label + fake_label)
    return images, labels

In [5]:
real_dir = "/kaggle/input/casia-dataset/CASIA1/Au"
fake_dir = "/kaggle/input/casia-dataset/CASIA1/Sp"
images, labels = load(real_dir, fake_dir)

In [6]:
normal_images = images.astype('float32') / 255
categories = keras.utils.to_categorical(labels, 2)

x_train, x_test, y_train, y_test = train_test_split(normal_images, categories, test_size=0.2, random_state=42)
#x_test, x_cv, y_test, y_cv = train_test_split(x_, y_, test_size=0.5, random_state=42)

In [6]:
def adjust_learning_rate(epochs):
  learning_rate = 1e-2
  if epochs > 160:
    learning_rate *= 5e-4
  elif epochs > 120:
    learning_rate *= 1e-3
  elif epochs > 80:
    learning_rate *= 5e-3
  elif epochs > 40:
    learning_rate *= 5e-2
  elif epochs >= 0:
    learning_rate *= 1e-1
  return learning_rate

In [7]:
def xceptionNet():
    img_input = Input(shape=(299, 299, 3))

    # Entry Flow
    x = Conv2D(32, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-3))(img_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-3))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    residual = Conv2D(128, (1, 1), strides=(2, 2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-2))(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(128, (3, 3), padding='same', )(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = SeparableConv2D(128, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # Block 3
    residual = Conv2D(256, (1, 1), strides=(2, 2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-2))(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(256, (3, 3), padding='same', depthwise_initializer='he_normal', pointwise_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = SeparableConv2D(256, (3, 3), padding='same', depthwise_initializer='he_normal', pointwise_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # Block 4
    residual = Conv2D(728, (1, 1), strides=(2, 2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-2))(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(728, (3, 3), padding='same', depthwise_initializer='he_normal', pointwise_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = SeparableConv2D(728, (3, 3), padding='same', depthwise_initializer='he_normal', pointwise_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    # Middle Flow
    for _ in range(8):
        residual = x
        x = Activation('relu')(x)
        x = SeparableConv2D(728, (3, 3), padding='same', depthwise_initializer='he_normal', pointwise_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        
        x = Activation('relu')(x)
        x = SeparableConv2D(728, (3, 3), padding='same', depthwise_initializer='he_normal', pointwise_initializer='he_normal')(x)
        x = BatchNormalization()(x)

        x = Activation('relu')(x)
        x = SeparableConv2D(728, (3, 3), padding='same', depthwise_initializer='he_normal', pointwise_initializer='he_normal')(x)
        x = BatchNormalization()(x)

        x = layers.add([x, residual])

    # Exit Flow
    residual = Conv2D(1024, (1, 1), strides=(2, 2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(1e-2))(x)
    residual = BatchNormalization()(residual)

    x = SeparableConv2D(728, (3, 3), padding='same', depthwise_initializer='he_normal', pointwise_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = SeparableConv2D(1024, (3, 3), padding='same', depthwise_initializer='he_normal', pointwise_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.add([x, residual])

    x = SeparableConv2D(1536, (3, 3), padding='same', depthwise_initializer='he_normal', pointwise_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = SeparableConv2D(2048, (3, 3), padding='same', depthwise_initializer='he_normal', pointwise_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    results = Dense(2, activation='softmax', kernel_initializer='he_normal')(x)

    model = Model(inputs=img_input, outputs=results, name='xception')
    return model


In [8]:
model = xceptionNet()
lr_scheduler = LearningRateScheduler(adjust_learning_rate)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=5e-6)
model_type = 'XceptionNet'
save_dir = os.path.join(os.getcwd(), 'saved_models') 
model_name = f'Casia1_{model_type}_model.{{epoch:03d}}.keras'
if not os.path.isdir(save_dir): 
    os.makedirs(save_dir) 
filepath = os.path.join(save_dir, model_name)
checkpoint = ModelCheckpoint(filepath=filepath, 
                              monitor='val_accuracy', 
                              verbose=1, 
                              save_best_only=True) 
callbacks = [checkpoint, lr_reducer, lr_scheduler]

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=adjust_learning_rate(0)),
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=64, epochs=200, validation_data=(x_test, y_test),
          shuffle=True, callbacks=callbacks)

Epoch 1/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.6657 - loss: 39.3465
Epoch 1: val_accuracy improved from -inf to 0.51594, saving model to /kaggle/working/saved_models/Casia1_XceptionNet_model.001.keras
22/22 ━━━━━━━━━━━━━━━━━━━━ 244s 6s/step - accuracy: 0.6680 - loss: 39.1690 - val_accuracy: 0.5159 - val_loss: 26.9369 - learning_rate: 0.0010
Epoch 2/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7622 - loss: 23.9659
Epoch 2: val_accuracy did not improve from 0.51594
22/22 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.7628 - loss: 23.8562 - val_accuracy: 0.5159 - val_loss: 16.6170 - learning_rate: 0.0010
Epoch 3/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7747 - loss: 14.7144
Epoch 3: val_accuracy did not improve from 0.51594
22/22 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.7752 - loss: 14.6533 - val_accuracy: 0.4841 - val_loss: 10.6743 - learning_rate: 0.0010
Epoch 4/200
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8385 - loss: 9.3891
Ep

In [8]:
#save_dir = os.path.join(os.getcwd(), 'saved_models')
#best_model_path = os.path.join(save_dir, "/kaggle/working/saved_models/Casia1_XceptionNet_model.108.keras")
#best_model = load_model(best_model_path)

In [7]:
Xception_path = os.path.join("/kaggle/input/xceptionnet/keras/default/1", "/kaggle/input/xceptionnet/keras/default/1/Casia1_XceptionNet_model.108.keras")
Xception = load_model(Xception_path)

In [8]:
y_pred = Xception.predict(x_test)
y_pred_class = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_class))

11/11 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step 
              precision    recall  f1-score   support

           0       0.92      0.95      0.94       178
           1       0.94      0.92      0.93       167

    accuracy                           0.93       345
   macro avg       0.93      0.93      0.93       345
weighted avg       0.93      0.93      0.93       345



In [ ]:
#import shutil
#shutil.rmtree("/kaggle/working/saved_models")


In [8]:
feature_extraction = Model(inputs=Xception.inputs, outputs=Xception.layers[-4].output)

In [9]:
x_feature_train = feature_extraction.predict(x_train)
x_feature_test = feature_extraction.predict(x_test)

43/43 ━━━━━━━━━━━━━━━━━━━━ 23s 202ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step 


In [10]:
x_feature_train.shape

(1376, 10, 10, 2048)

In [11]:
timesteps = 5
x_train_reshaped = np.expand_dims(x_feature_train, axis=1) 
x_train_reshaped = np.tile(x_train_reshaped, (1, timesteps, 1, 1, 1))

In [12]:
x_train_reshaped.shape

(1376, 5, 10, 10, 2048)

In [13]:
x_test_reshapped = np.expand_dims(x_feature_test, axis=1)
x_test_reshapped = np.tile(x_test_reshapped, (1, timesteps, 1, 1, 1))
x_test_reshapped.shape

(345, 5, 10, 10, 2048)

In [17]:
model_type = 'LSTM'
save_dir = os.path.join(os.getcwd(), 'saved_models') 
model_name = f'LSTM_{model_type}_model.{{epoch:03d}}.keras'
if not os.path.isdir(save_dir): 
    os.makedirs(save_dir) 
filepath = os.path.join(save_dir, model_name)
filepath = os.path.join(save_dir, model_name)
checkpoint = ModelCheckpoint(filepath=filepath, 
                              monitor='val_accuracy', 
                              verbose=1, 
                              save_best_only=True) 

conv_lstm = Sequential()
conv_lstm.add(TimeDistributed(Conv2D(128, (1, 1), activation='relu'), input_shape=(5, 10, 10, 2048)))
conv_lstm.add(ConvLSTM2D(filters=32, kernel_size=(3, 3), strides=(2, 2), 
                       kernel_initializer='he_normal',
                      return_sequences=False, input_shape=(5, 10, 10, 2048)))

conv_lstm.add(Flatten())
conv_lstm.add(Dense(256, activation='relu'))
conv_lstm.add(Dropout(0.2))
conv_lstm.add(Dense(2, activation='softmax'))
conv_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
conv_lstm.fit(x_train_reshaped, y_train, batch_size=8, epochs=10, validation_data=(x_test_reshapped, y_test),
             shuffle=True, callbacks=checkpoint)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
171/172 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.9772 - loss: 0.0503
Epoch 1: val_accuracy improved from -inf to 0.92174, saving model to /kaggle/working/saved_models/LSTM_LSTM_model.001.keras
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 58ms/step - accuracy: 0.9774 - loss: 0.0499 - val_accuracy: 0.9217 - val_loss: 0.6364
Epoch 2/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 1.0000 - loss: 4.0295e-05
Epoch 2: val_accuracy improved from 0.92174 to 0.92464, saving model to /kaggle/working/saved_models/LSTM_LSTM_model.002.keras
172/172 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 1.0000 - loss: 4.0257e-05 - val_accuracy: 0.9246 - val_loss: 0.6815
Epoch 3/10
169/172 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 1.0000 - loss: 1.2319e-05
Epoch 3: val_accuracy did not improve from 0.92464
172/172 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 1.0000 - loss: 1.2508e-05 - val_accuracy: 0.9246 - val_loss: 0.6991
Epoch 4/10
169/172 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 1

In [20]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
lstm_path = os.path.join(save_dir, "/kaggle/working/saved_models/LSTM_LSTM_model.002.keras")
lstm = load_model(lstm_path)

In [21]:
prediction = lstm.predict(x_test_reshapped, batch_size=8)
prediction_class = np.argmax(prediction, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, prediction_class))

44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step
              precision    recall  f1-score   support

           0       0.90      0.96      0.93       178
           1       0.95      0.89      0.92       167

    accuracy                           0.92       345
   macro avg       0.93      0.92      0.92       345
weighted avg       0.93      0.92      0.92       345

